# Wave with periodic bc with the scheme from Palma

We solve the wave equation with Dirichlet boundary conditions (homogeneous) up to $T = 2L$ where $L$ is the space size of the integrating region. The data is reflection symmetric, $u_0(x) = u_0(L-x)$, $\partial_t u(x,t=0) = 0$. This way at the last point the solution repeats itself and we can measure the error very well.

In [ ]:
using ComponentArrays
using Distributions
#using WGLMakie
using Lux
using LuxCUDA
using OptimizationOptimJL
using Random
using UnPack
using Zygote
#using CairoMakie  # Backend que funciona en notebooks
using StatsBase
using Revise
using JLD2
using Plots
using LinearAlgebra
#import NaNMath



I put some functions here.

In [ ]:
includet("neural_tools.jl")

In [ ]:
@load "../../wave_Dir_1.jld2" config Θ st losses

NN = create_chain(config)

In [ ]:
# -------------------------------------------------------------------
# Second order wave equation (onda 1D): u_tt - c^2 u_xx = 0
# -------------------------------------------------------------------
#wave_equation(∂2u_∂x2, ∂2u_∂t2, c) = ∂2u_∂t2 .- (c^2) .* ∂2u_∂x2

In [ ]:
plot(losses, label = "Loss", title = "loss vs iterations")


In [ ]:

t = 2 #config[:tmax]
xs = reshape(collect(range(config[:xmin], config[:xmax], length=200)), 1, :)
t_fix = reshape(fill(t, length(xs)), 1, :)
#sol_p = [calculate_Dirichlet_f(xs, t_fix, NN, Θ, st) for x in xs]
sol_p = [calculate_Dirichlet_f(xs[:,i], t_fix[:,i], NN, Θ, st)[1] for i in 1:length(xs[1,:])]


plot(xs[1,:],sol_p, label = "t=$t", title = "solution")
#lines!(xs[1,:],bump.(xs[1,:], config[:x0], config[:x1], config[:p], config[:A]), label = "initial condition")
plot!(xs[1,:],config[:A]*(xs[1,:] .- config[:xmin]).^4 .* (xs[1,:] .- config[:xmax]).^4/((config[:xmax]-config[:xmin])/2)^8, label = "initial condition", linestyle = :dash)     


The error looks like a polynomial of order 10.

In [ ]:
plot(title = "solution")
t = 2.0
xs = reshape(collect(range(config[:xmin], config[:xmax], length=200)), 1, :)
t_fix = reshape(fill(t, length(xs)), 1, :)
#sol_p = [calculate_Dirichlet_f(xs, t_fix, NN, Θ, st) for x in xs]
sol_p = [calculate_Dirichlet_f(xs[:,i], t_fix[:,i], NN, Θ, st)[1] for i in 1:length(xs[1,:])]

plot!(xs[1,:],sol_p .- (config[:A]*(xs[1,:] .+ config[:xmin]).^4 .* (xs[1,:] .- config[:xmax]).^4)*2^8, label = "t=0")
#lines!(xs[1,:],bump.(xs[1,:], config[:x0], config[:x1], config[:p], config[:A]), label = "initial condition")
#lines!(xs[1,:],sol_p .-config[:A]*(xs[1,:] .- config[:xmin]).^4 .* (xs[1,:] .- config[:xmax]).^4, label = "initial condition", linestyle = :dash)     


In [ ]:
norm(sol_p .- (config[:A]*(xs[1,:] .+ config[:xmin]).^4 .* (xs[1,:] .- config[:xmax]).^4)*2^8)/norm(config[:A]*(xs[1,:] .- config[:xmin]).^4 .* (xs[1,:] .- config[:xmax]).^4)/sqrt(length(xs[1,:]))

Norm Error = 0.02606418374105666, When loss = -3.5 

In [ ]:
norm([1,1,1])/sqrt(length([1,1,1]))